# Basic BoW model 

In [1]:
from collections import defaultdict
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from data import get_data

In [2]:
def cleanup(x):
    # removes punctuation
    x = re.sub(r'[.,"!#/]+', ' ', x, flags=re.MULTILINE)
    # removs \\n
    x = re.sub(r'\\n', ' ', x, flags=re.MULTILINE)
    # removes retweets (e.g. @FER)
    x = re.sub(r'@\w+', '', x, flags=re.MULTILINE)

    return x

In [3]:
def vectorize_X(tweet, vocab):
    # obsolete 
    # tweet = re.sub(r'[.,"!#]+', ' ', tweet, flags=re.MULTILINE)
    #tweet = re.sub(r'@\w+', ' ', tweet, flags=re.MULTILINE)

    # splits tags into 2, e.g. #happy -> #, happy
    # TODO: tags could be useful for classification
    tweet_toks = word_tokenize(tweet)
    X_ = []
    for w in vocab:
        X_.append(1 if w in tweet_toks else 0)

    return X_

In [4]:
def create_vocab(tweets):
    vocab = set()
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    #stemmer = SnowballStemmer()
    
    clean_tweets = [cleanup(t) for t in tweets]
    
    for t in clean_tweets:
        for w in word_tokenize(t):
            vocab.add(w.strip())
    
    vocab -= set(stop_words)
        
    return sorted(vocab)

In [5]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [7]:
X, y = get_data('data/2018-E-c-En-dev.txt')

vocab = create_vocab(X)

vectorizer = CountVectorizer(tokenizer=word_tokenize, stop_words='english', vocabulary=vocab)
clf = LinearSVC(random_state=42)

print(len(X), len(y))
estimators = [('vectorize', vectorizer), ('clf', clf)]
pipe = Pipeline(memory=None, steps=estimators).fit(X, y)
#pipe.predict("asdf")

886 886


ValueError: bad input shape (886, 11)

In [50]:
c = OneVsRestClassifier(LinearSVC(random_state=42)).fit(X, y)

c.predict([vectorize_X("oh hidden revenge and anger...I rememberthe time,she rebutted you", vocab)])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])